In [1]:
from ultralytics import YOLO
import cv2
from posehead.PoseHead import PoseHead
import torch

训练

In [ ]:
# 加载模型
# model = YOLO("yolov8n.yaml")  # 从头开始构建新模型
model = YOLO("yolov8n-pose.pt")  # 加载预训练模型（建议用于训练）
# model = YOLO('runs/detect/train5/weights/best.pt')    # 加载训练完成的模型
# 使用模型
model.train(data="bird.yaml", epochs=30)  # 训练模型

预测图片

In [11]:
# Load a model
# model = YOLO('yolov8n-pose.pt')  # load an official model
model = YOLO('runs/pose/train/weights/best.pt')  # load a custom model

# Predict with the model
results = model('imgs/bird2.jpg')  # predict on an image
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    result.show()  # display to screen


image 1/1 D:\个人\大学文件\python\yolov8\imgs\bird2.jpg: 512x640 1 bird, 54.9ms
Speed: 5.0ms preprocess, 54.9ms inference, 2.0ms postprocess per image at shape (1, 3, 512, 640)


预测视频

In [ ]:
class_names = {0: 'stop', 1: 'fly'}
# model = YOLO('yolov8n-pose.pt')
model = YOLO('runs/pose/train-综合/weights/best.pt')
posehead = PoseHead().to(torch.device('cuda'))
posehead.load_state_dict(torch.load("posehead/posehead.pt"))
posehead.eval()

video_path = 'imgs/birds2.mp4'
cap = cv2.VideoCapture(video_path)

# 定义视频保存的参数
output_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
output_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
output_filename = "output_video2_综合.mp4"

# 创建 VideoWriter 对象
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(output_filename, fourcc, fps, (output_width, output_height))
# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        results = model(frame)
        # Visualize the results on the frame
        for i, result in enumerate(results):
            if i != 0:
                annotated_frame = annotated_frame.plot()
            else:
                annotated_frame = results[0].plot()
            if len(result.boxes) != 0:
                output = posehead(result.keypoints.xyn)
                predicted_class = output[0].argmax()
                class_name = class_names[predicted_class.item()]
                x1, y1, x2, y2 = result.boxes.xyxy[0][:4]  # 获取边界框的坐标信息
                cv2.putText(annotated_frame, class_name, (int(x1) + 20, int(y2) + 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)  # 在左上角写上类名

        cv2.imshow("YOLOv8 Inference", annotated_frame)
        video_writer.write(annotated_frame)
        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        break

cap.release()
video_writer.release()
cv2.destroyAllWindows()


0: 640x640 1 bird, 16.5ms
Speed: 8.0ms preprocess, 16.5ms inference, 258.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 bird, 15.0ms
Speed: 4.0ms preprocess, 15.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 bird, 15.0ms
Speed: 6.0ms preprocess, 15.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 bird, 14.9ms
Speed: 3.0ms preprocess, 14.9ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 bird, 15.0ms
Speed: 4.0ms preprocess, 15.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 bird, 15.0ms
Speed: 3.0ms preprocess, 15.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 bird, 15.0ms
Speed: 4.0ms preprocess, 15.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 bird, 16.0ms
Speed: 5.0ms preprocess, 16.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 64

D:\python\lib\site-packages\torch\nn\functional.py:5476: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


验证

In [2]:
from ultralytics import YOLO

# Load a model
# model = YOLO('yolov8n-pose.pt')  # load an official model
model = YOLO('runs/pose/train8/weights/best.pt')  # load a custom model

# Validate the model
metrics = model.val()  # no arguments needed, dataset and settings remembered
metrics.box.map    # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps   # a list contains map50-95 of each category

Ultralytics YOLOv8.1.18 🚀 Python-3.9.1 torch-2.2.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1650 Ti, 4096MiB)
YOLOv8n-pose summary (fused): 187 layers, 3136820 parameters, 0 gradients, 8.6 GFLOPs
                   all        456       1449      0.997      0.997      0.995       0.98      0.986      0.987      0.989      0.633
Speed: 0.3ms preprocess, 6.4ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs\pose\val


val: Scanning D:\个人\大学文件\python\yolov8\datasets\bird\labels\val.cache... 456 images, 0 backgrounds, 0 corrupt: 100%|██████████| 456/456 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [00:12<00:00,  2.40it/s]


array([    0.97981])